sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs 
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
 

In [2]:
DRIVER_PATH = 'C:/Users/thisi/Documents/GitHub/pq-scraper/chromedriver_win32/chromedriver.exe'
SEARCH_TERM = 'enter search term here' # can be blank
PARLIAMENT_INDEX = 1 # legislative assembly = 1, 1st parliament = 2, etc etc. -1 to ignore this field.


In [3]:
driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://sprs.parl.gov.sg/search/home' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 
 
# Get search box and fill it up 

search = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[1]/input')
search.send_keys(SEARCH_TERM) 
 
# Uncomment following two lines to only search in titles # 
#checkbox = driver.find_element_by_xpath('//*[@id="divmpscreen2"]/div[2]/div[1]/div/label/input')
#checkbox.click() 

if PARLIAMENT_INDEX != -1:
    session = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[3]/select/option[{PARLIAMENT_INDEX}]')
    session.click() 
 
# Find submit element and press enter

submit = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[3]/div/button[2]') 
submit.send_keys(Keys.RETURN) 

print('Search parameters submitted.') 

# Switch window and check for number of search results 

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(20) 
time.sleep(2) 

try:
    num_res_label = driver.find_element(by=By.CLASS_NAME, value='showingResults')
except: 
    print('Error when finding num_res_label')
    
num_res_text = num_res_label.get_attribute('innerHTML')
    
if 'No' in num_res_text:
    print('no results found') # but site is quite erratic, may take a few tries to get results (idk why)
else:
    num_results = int(num_res_text[num_res_text.rfind(' ')+1:])
    num_pages = math.ceil(num_results / 20)
    print(f'{num_results} results found spanning {num_pages} page(s)')

C:\Users\thisi\AppData\Local\Temp\ipykernel_17212\2453240245.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Search parameters submitted.
16 results found spanning 1 page(s)


In [4]:
docs_saved = 0
pages_seen = 0

if not os.path.exists('scraped_content'):
    os.makedirs('scraped_content')
    
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

while (True):
    driver.switch_to.window(driver.window_handles[1])                  

    elems = driver.find_elements(by=By.XPATH, value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/table/tbody')
    links = list(map(lambda elem: elem.find_elements(by=By.XPATH, value='.//tr[1]/td[2]/a')[0], elems))
    
    print(f'{len(links)} links on this page')
    
    for link in links:
        y_height = 0
        while not element_completely_viewable(driver, link):
            driver.execute_script(f"window.scrollTo(0, {y_height})") 
            y_height += 100
            time.sleep(0.5)
        
        link.click()
            
        # Switch to page with content and get URL name
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[2])         
        item_key = driver.current_url.split('/')[-1]         
        item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 

        # Write out each page source as a file         
        with open('scraped_content/' + item_key + '.txt', encoding = 'utf-8', mode = 'w+') as file:             
            file.write(driver.page_source)    
        docs_saved += 1
        print(f'Saved {docs_saved} / {num_results} docs')

        # Close tab         
        driver.close()
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)
    
    if docs_saved == num_results:
        break
    elif docs_saved == 20:
        next_page = driver.find_element_by_xpath('//*[@id="searchResults"]/div[3]/section/ul/li[1]/a/i')
    else:        
        next_page = driver.find_element_by_xpath('//*[@id="searchResults"]/div[3]/section/ul/li[3]/a/i')     
    
    next_page.click()
    pages_seen += 1
    print(f'Seen {pages_seen} / {num_pages} pages')

    # Sleep momentarily because next page takes a while to load     
    time.sleep(2) 


16 links on this page
Saved 1 / 16 docs
Saved 2 / 16 docs
Saved 3 / 16 docs
Saved 4 / 16 docs
Saved 5 / 16 docs
Saved 6 / 16 docs
Saved 7 / 16 docs
Saved 8 / 16 docs
Saved 9 / 16 docs
Saved 10 / 16 docs
Saved 11 / 16 docs
Saved 12 / 16 docs
Saved 13 / 16 docs
Saved 14 / 16 docs
Saved 15 / 16 docs
Saved 16 / 16 docs


In [5]:
exit()

In [6]:
# old codee here.
# Nested for loop to click through all search results 
for page in range(num_pages):         
    # This assumes that 20 search results are returned, which are 1-indexed in the xpaths     
    for item in range (1, 21):
        # Switch to search results page         
        driver.switch_to.window(driver.window_handles[1])                  
        # Get element to click on, to see each individual page with content         
        # Last page will have fewer than 20 elements, so need to handle this exception         
        try:             
            elem = driver.find_element_by_xpath('//*[@id="searchResults"]/table/tbody[{}]/tr[1]/td[2]/a'.format(item))
            elem.click()     
            
            # Switch to page with content and get URL name         
            driver.switch_to.window(driver.window_handles[2])         
            item_key = driver.current_url.split('/')[-1]         
            item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 

            # Append result to dictionary for later processing         
            res_dict[item_key] = driver.page_source  

            # Write out each page source as a file         
            with open('scraped_content/' + item_key + '.txt', encoding = 'utf-8', mode = 'w+') as file:             
                file.write(driver.page_source)                  

            # Close tab         
            driver.close()              
        
        except:             
            break   
                
        # Switch back to search results tab     
        driver.switch_to.window(driver.window_handles[1])          
        
        # Click on next page once 20 results have been saved     
        # Next page button changes after first 20 results are shown, hence the need for enclosing the element xpath in a try block     
        try:         
            next_page = driver.find_element_by_xpath('//*[@id="searchResults"]/div[3]/section/ul/li[3]/a/i')     
        except:         
            next_page = driver.find_element_by_xpath('//*[@id="searchResults"]/div[3]/section/ul/li[1]/a/i')     
        next_page.click()          
        
        # Sleep momentarily because next page takes a while to load     
        time.sleep(2) 

# Check that all results are stored 
assert len(res_dict.keys()) == int(res[-1]), "It looks like not all the results were stored!" 

C:\Users\thisi\AppData\Local\Temp\ipykernel_17212\3422452015.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  elem = driver.find_element_by_xpath('//*[@id="searchResults"]/table/tbody[{}]/tr[1]/td[2]/a'.format(item))


NameError: name 'res_dict' is not defined